***
<center><h1>Face Rhythm</h1></center>

***

<table><tr>
<td> <img src="https://images.squarespace-cdn.com/content/5688a31305f8e23aa2893502/1614723283221-5Z5038AT7Y6KCOM2PIU4/Screenshot+from+2021-03-02+17-05-12.png?content-type=image%2Fpng" style="height: 200px"> </td>
<td> <img src="https://images.squarespace-cdn.com/content/5688a31305f8e23aa2893502/1614723856628-J89PYYSF7K7JATE2KMF9/Screenshot+from+2021-03-02+17-23-46.png?format=300w&content-type=image%2Fpng" style="height: 200px"> </td>
<td> <img src="https://images.squarespace-cdn.com/content/5688a31305f8e23aa2893502/1614723931026-OORV0RAPZNWV3R8TBOXB/Screenshot+from+2021-03-02+17-25-11.png?format=300w&content-type=image%2Fpng" style="height: 200px"> </td>
<td> <img src="https://images.squarespace-cdn.com/content/5688a31305f8e23aa2893502/1614724055033-O3GBEF1D9MULFZKI2IUJ/Screenshot+from+2021-03-02+17-27-10.png?format=300w&content-type=image%2Fpng" style="height: 200px"> </td>
<td> <img src="https://images.squarespace-cdn.com/content/5688a31305f8e23aa2893502/1614723378405-WXN74ZTT1KYZUQGDM07X/face_rhythm_banner2.png?format=1000w&content-type=image%2Fpng" style="height: 200px"> </td>
</tr></table>

***

##### Notebook Shortcuts
- **[Notebook Setup](#Notebook-Setup)**: Prepare all the necessary config files and folders
- **[Set ROI](#Set-ROI)**: Set the ROI for the analysis
- **[Run Optic Flow](#Run-Optic-Flow)**: Run the optic flow analysis
- **[Clean Optic Flow](#Clean-Optic-Flow)**: Optic flow post-processing
- **[Convolutional Dimensionality Reduction](#Convolutional-Dimensionality-Reduction)**: Convolutional Dimensionality Reduction
- **[Analysis](#Analysis)**: Decompose and Analyze the optic flow data in many ways
- **[Comparisons](#Comparisons)**: Compare Face Rhythm to some peer algorithms

***

# Tips on running this notebook:
In theory it would be nice if you could just enter the path of the video(s) and just let it run all the way through. In practice, there are a few hoops to jump through
- Run the Notebook Setup Block (two blocks below this one). This should pretty much always be done, even if you are loading precomputed file from disk instead of calculating them. This step loads in some useful meta data used throughout.
- Even if you are restarting at a specific point in your analysis, run your Setup Block then head down to your current analysis step cell 

# Project Directory Organization
------------

    Project Directory
    ├── config.yaml           <- Configuration parameters to run each module in the pipeline. Dictionary.
    ├── run_info.json         <- Output information from each module. Dictionary.
    │
    ├── run_data              <- Output data from each module.
    │   ├── Dataset_videos.h5 <- Output data from Dataset_videos class. Contains metadata about the videos.
    │   ├── ROIs.h5           <- Output data from ROIs class. Contains ROI masks.
    │   ├── PointTracker.h5   <- Output data from PointTracker class. Contains point tracking data.
    |   ├── VQT_Analyzer.h5   <- Output data from VQT_Analyzer class. Contains spectral decomposition data.
    │   ├── TCA.h5            <- Output data from TCA class. Contains TCA decomposition data.
    │   
    └── visualizations        <- Output visualizations.
        ├── factors_rearranged_[frequency].png  <- Example of a rearranged factor plot.
        └── point_tracking_demo.avi             <- Example video.



***
<center><h1>Notebook Setup</h1></center>

***

### Creates config and locates videos

**Crucially, always run this first cell every time you run this notebook.**

Also, generally make sure to read through the config parameters before running.

The Project path is the path to a folder (existing or not) where we will store our derived files. I recommend creating a project folder and then copying this notebook into that folder.
The Video path is the path to a folder containing videos. 
The run name will determine the name of the config. You might create multiple configs if you want to re-run the same data with slightly different parameters

Previous face rhythm users might be familiar with the 'sessions' structure. Some users will want to run multiple sessions through Face Rhythm at the same time. If that's you, then read the docs to see what parameters to change:
https://face-rhythm.readthedocs.io/

If you did everything according to the readme, you should see that the video_path currently points to a folder containing one sample video in the testing folder. Give this a try!

In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2
import face_rhythm as fr

from pprint import pprint
from pathlib import Path

import cv2

import numpy as np
import torch
import matplotlib.pyplot

fr.util.system_info(verbose=True);

== Operating System ==: uname_result(system='Linux', node='rich-L-CLASS', release='5.15.0-134-generic', version='#145~20.04.1-Ubuntu SMP Mon Feb 17 13:27:16 UTC 2025', machine='x86_64')
== CPU Info ==: {'n_cores': 36, 'brand': 'Intel(R) Core(TM) i9-10980XE CPU @ 3.00GHz'}
== RAM ==: svmem(total=269924495360, available=257510146048, percent=4.6, used=9983664128, free=214168133632, active=8855154688, inactive=37806112768, buffers=19860213760, cached=25912483840, shared=79273984, slab=8381267968)
== GPU Info ==: {0: {'id': 0, 'uuid': 'GPU-361d808e-1136-2078-83bf-4290bf948f25', 'load': 0.38, 'memoryUtil': 0.045817057291666664, 'memoryTotal': 24576.0, 'memoryUsed': 1126.0, 'memoryFree': 23100.0, 'driver': '535.183.01', 'name': 'NVIDIA GeForce RTX 3090', 'serial': '1324220011997', 'display_mode': 'Enabled', 'display_active': 'Enabled', 'temperature': 51.0}}
== Conda Environment ==: fr
== Python Version ==: 3.11.11
== GCC Version ==: 9.4.0
== PyTorch Version ==: 2.6.0+cu124
== CUDA Version ==

In [2]:
# Change these to your desired project path and video path
# Assuming that your notebook is running  in your project directory, 
# then the project directory is current working directory
# Also, we have some test videos in the face_rhythm repo, so you can set the video directory there for testing
directory_project = '/media/rich/bigSSD/analysis_data/face_rhythm/musall_data/2708_PuffyPenguin_20230524_151723'
directory_videos  = '/media/rich/bigSSD/other_lab_data/Musall_data/2708__20250321/2708/20230524_151723/'

filename_strMatch = '.*avi'  ## You can use regular expressions to search and match more complex strings

In [3]:
path_config, path_run_info, directory_project = fr.project.prepare_project(
    directory_project=directory_project,
    overwrite_config=False,  ## WARNING! CHECK THIS.
    update_project_paths=True,
    mkdir=True,    
    initialize_visualization=True,    
    verbose=2,
)
figure_saver = fr.util.Figure_Saver(
    path_config=path_config,
    formats_save=['png'],
    kwargs_savefig={'bbox_inches': 'tight', 'pad_inches': 0.1, 'transparent': True, 'dpi': 300},
    overwrite=True,
    verbose=2,
)

Initializing cv2.imshow
FR: Found config.yaml file at /media/rich/bigSSD/analysis_data/face_rhythm/musall_data/2708_PuffyPenguin_20230524_151723/config.yaml
FR: Updating project paths in config.yaml file at /media/rich/bigSSD/analysis_data/face_rhythm/musall_data/2708_PuffyPenguin_20230524_151723/config.yaml


# Prepare video data for point tracking

In [4]:
paths_videos = fr.helpers.find_paths(
    dir_outer=directory_videos,
    reMatch=filename_strMatch,  ## string to use to search for files in directory. Uses regular expressions!
    depth=0,  ## how many folders deep to search
)

pprint(paths_videos, width=1000)

['/media/rich/bigSSD/other_lab_data/Musall_data/2708__20250321/2708/20230524_151723/2708_PuffyPenguin_20230524_151723_cam1_00000000.avi']


Make a `BufferedVideoReader` object for reading video file data

In [5]:
videos = fr.helpers.BufferedVideoReader(
#     video_readers=data.videos, 
    paths_videos=paths_videos,
    buffer_size=1000, 
    prefetch=1, 
    posthold=1,
    method_getitem='by_video',
    verbose=1,
)

Make a `Dataset_videos` object for referencing the raw video data

In [6]:
data = fr.data_importing.Dataset_videos(
    bufferedVideoReader=videos,
#     paths_videos=paths_videos,
    contiguous=False,
#     frame_rate_clamp=240,
    verbose=2,
);

printing this line helps the bufferedVideoReader object load properly.   <class 'face_rhythm.helpers.BufferedVideoReader'> <class 'type'> <class 'type'> True


Save the `Dataset_videos` object in the 'analysis_files' project folder

In [7]:
data.save_config(path_config=path_config, overwrite=True, verbose=1)
data.save_run_info(path_config=path_config, overwrite=True, verbose=1)
data.save_run_data(path_config=path_config, overwrite=True, verbose=1)

FR Warning: Overwriting field. 'Dataset_videos' already in config.yaml.
FR Warning: Overwriting field. 'Dataset_videos' is already a field in the run_info.json dictionary.
FR WARNING: path_run_data file is expected to be named 'Dataset_videos.h5' if it is part of a project. Please make sure this is correct.
FR Warning: Overwriting file. File: /media/rich/bigSSD/analysis_data/face_rhythm/musall_data/2708_PuffyPenguin_20230524_151723/analysis_files/Dataset_videos.h5 already exists.


# Define ROIs

Either select new ROIs (`select_mode='gui'`) and provide an exampleImage, or import existing ROIs (`path_file=path_to_ROIs.h5_file`).\
**Typically, you should make 1 or 2 ROIs. The first defining where the face points should be and the second for cropping the frame.**

In [11]:
exampleImage = data[0][0]
# %matplotlib notebook
rois = fr.rois.ROIs(
    select_mode='gui',
    exampleImage=exampleImage,
#     select_mode='file',
#     path_file=r'/media/rich/bigSSD/other_lab_data/Koyama_lab/zfish_head_movie_for_Bernardo/darkfield_FR_20240624/analysis_files/ROIs.h5',
    verbose=2
)

FR: Initializing GUI...


<IPython.core.display.Javascript object>

Button(description='Confirm ROI', style=ButtonStyle())

Button(description='New ROI', style=ButtonStyle())

Make a point grid out of the first ROI

In [9]:
rois.make_points(rois=rois[0], point_spacing=8)
# rois.make_points(rois=rois[0], point_spacing=13) if rois.point_positions is None else None

IndexError: list index out of range

Save the `ROIs` object in the 'analysis_files' project folder

In [ ]:
rois.save_config(path_config=path_config, overwrite=True, verbose=1)
rois.save_run_info(path_config=path_config, overwrite=True, verbose=1)
rois.save_run_data(path_config=path_config, overwrite=True, verbose=1)

visualize the ROIs

In [ ]:
rois.plot_rois(exampleImage)

# Point Tracking

Prepare `PointTracker` object.\
Set `visualize_video` to **`True`** to tune parameters until they look appropriate, then set to **`False`** to run the full dataset through at a much faster speed.

Key parameters:
- `point_spacing`: distance between points. Vary so that total number of points is appropriate.
- `mesh_rigidity`: how rigid the mesh elasticity is. Vary so that points track well without drift.
- `relaxation`: how quickly the points relax back to their home position. Vary so that points track well without dift.
- `kwargs_method > winSize`: the spatial size of the optical flow calculation. Smaller is better but noisier, larger is less accurate but more robust to noise.
- `params_outlier_handling > threshold_displacement`: point displacements above this value will result in freezing of the points.

In [ ]:
pt = fr.point_tracking.PointTracker(
    visualize_video=False,
    buffered_video_reader=videos,
    point_positions=rois.point_positions,
    rois_masks=[rois[1]],
    contiguous=True,
    params_optical_flow={
        "method": "lucas_kanade",
        "mesh_rigidity": 0.05,
        "mesh_n_neighbors": 24,
        "relaxation": 0.001,
        "kwargs_method": {
            "winSize": [25, 25],
            "maxLevel": 2,
            "criteria": [cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 2, 0.00003],
        },        
    },
    params_visualization={
        'alpha': 0.4,
        'point_sizes': 3,
    },
    params_outlier_handling = {
        'threshold_displacement': 150,  ## Maximum displacement between frames, in pixels.
        'framesHalted_before': 40,  ## Number of frames to halt tracking before a violation.
        'framesHalted_after': 40,  ## Number of frames to halt tracking after a violation.
    },
    idx_start=0,
    verbose=2,
)

Perform point tracking

In [ ]:
pt.track_points()

Save the `PointTracker` object in 'analysis_files' project directory.\
Using compression can reduce file sizes slightly but is very slow.

In [ ]:
pt.save_config(path_config=path_config, overwrite=True, verbose=1)
pt.save_run_info(path_config=path_config, overwrite=True, verbose=2)
pt.save_run_data(path_config=path_config, overwrite=True, use_compression=False, verbose=1)

Clear some memory if needed. Optional.

In [ ]:
pt.cleanup()

Load the `PointTracker` data as a dictionary

In [ ]:
pt_data = fr.h5_handling.simple_load(str(Path(directory_project) / 'analysis_files' / 'PointTracker.h5'), return_dict=True, verbose=True)

# Spectral Analysis

Prepare `VQT_Analyzer` object.

Key parameters:
- `Q_lowF`:  Quality of the lowest frequency band of the spectrogram. Q value is number of oscillation periods.
- `Q_highF`: Quality of the highest frequency band...
- `F_min`: Lowest frequency band to use.
- `F_max`: Highest frequency band to use.
- `downsample_factor`: How much to downsample the spectrogram by in time.
- `return_complex`: Whether or not to return the complex spectrogram. Generally set to False unless you want to try something fancy.

In [ ]:
%matplotlib inline
Fs = fr.util.load_run_info_file(path_run_info)['Dataset_videos']['frame_rate']

spec = fr.spectral_analysis.VQT_Analyzer(
    params_VQT = {
        'Fs_sample': Fs, 
        'Q_lowF': 2.5, 
        'Q_highF': 5, 
        'F_min': 0.5, 
#         'F_max': data.frame_rate//2, 
        'F_max': 60, 
        'n_freq_bins': 60, 
        'win_size': 4501, 
        'plot_pref': True, 
        'downsample_factor': 8, 
        'padding': 'valid',
        'DEVICE_compute': fr.helpers.set_device(use_GPU=True), 
        'batch_size': 10,
        'return_complex': False, 
        'progressBar': True,
    },
    normalization_factor=0.90,
    spectrogram_exponent=1.0,
    one_over_f_exponent=0.5,
    verbose = 2,
)

Look at a demo spectrogram of a single point.\
Specify the point with the `idx_point` and `name_points` fields.\
Note that the `pt_data['points_tracked']` dictionary holds subdictionaries withe numeric string names (ie `['0'], ['1']`) for each video.

In [ ]:
%matplotlib notebook
spec_demo, xAxis_demo, freqs_demo = spec.demo_transform(
    points_tracked=pt_data['points_tracked'],
    point_positions=pt_data['point_positions'],
    idx_point=500,
    name_points='0',
    plot=True,
);

Generate spectrograms

In [ ]:
spec.transform_all(
    points_tracked=pt_data['points_tracked'],
    point_positions=pt_data['point_positions'],
)

Save the `VQT_Analyzer` object in 'analysis_files' project directory.\
Using compression can reduce file sizes slightly but is very slow.

In [ ]:
spec.save_config(path_config=path_config, overwrite=True, verbose=1)
spec.save_run_info(path_config=path_config, overwrite=True, verbose=1)
spec.save_run_data(path_config=path_config, overwrite=True, use_compression=False, verbose=1)

Clear some memory if needed. Optional.

In [ ]:
spec.cleanup()

Load the `VQT_Analyzer` data as a dictionary

In [ ]:
spec_data = fr.h5_handling.simple_load(str(Path(directory_project) / 'analysis_files' / 'VQT_Analyzer.h5'), return_dict=True, verbose=True)

# Decomposition

Prepare `TCA` object, and then rearrange the data with the `.rearrange_data` method.

Key parameters for `.rearrange_data`:
- `names_dims_array`:  Enter the names of the dimensions of the spectrogram. Typically these are `'xy', 'points', 'frequency', 'time'`.
- `names_dims_concat_array`: Enter any dimensions you wish to concatenate along other dimensions. Typically we wish to concatenate the `'xy'` dimension along the `'points'` dimension, so we make a list containing that pair as a tuple: `[('xy', 'points')]`.
- `concat_complexDim`: If your input data are complex valued, then this can concatenate the complex dimension along another dimension.
- `name_dim_dictElements`: The `data` argument is expected to be a dictionary of dictionaries of arrays, where the inner dicts are trials or videos. This is the name of what those inner dicts are. Typically `'trials'`.

In [ ]:
# spectrograms = spec_data['spectrograms']
spectrograms = {key: np.abs(val) for key,val in list(spec_data['spectrograms'].items())[:]}

tca = fr.decomposition.TCA(verbose=2)

tca.rearrange_data(
    data=spectrograms,
    names_dims_array = ['xy', 'points', 'frequency', 'time'],
    names_dims_concat_array = [('xy', 'points')],
    concat_complexDim=False,
    name_dim_concat_complexDim='time',
    name_dim_dictElements = 'session',
    method_handling_dictElements = 'separate',
    name_dim_concat_dictElements = 'time',
    idx_windows=None,
    name_dim_array_window='time',
)

Optionally, you can normalize along a specified axis

In [ ]:
tca.normalize_data(
    mean_subtract=False,
    std_divide=True, 
    dim_name='time', 
)

Fit TCA model.

There are a few methods that can be used:
- `'CP_NN_HALS'`: non-negative CP decomposition using the efficient HALS algorithm. This should be used in most cases.
- `'CP'`: Standard CP decomposition. Use if input data are not non-negative (if you are using complex valued spectrograms or similar).
- `'Randomized_CP'`: Randomized CP decomposition. Allows for large input tensors. If you are using huge tensors and you are memory constrained or want to run on a small GPU, this is your only option.

If you have and want to use a CUDA compatible GPU:
- Set `DEVICE` to `'cuda'`
- GPU memory can be saved by setting `'init'` method to `'random'`. However, fastest convergence and highest accuracy typically come from `'init': 'svd'`.

In [ ]:
tca.fit(
    method='CP_NN_HALS',
#     method='CP',
    params_method={
        'rank': 10, 
        'n_iter_max': 1000, 
        'init': 'svd', 
        'svd': 'truncated_svd', 
        'tol': 1e-09, 
#         'nn_modes': [0,1], 
        'normalize_factors': False,
        'verbose': True, 
    },
    DEVICE=fr.helpers.set_device(use_GPU=True),
    verbose=2,
)

Rearrange the factors.\
You can undo the concatenation that was done during `.rearrange_data`

In [ ]:
tca.order_factors_by_EVR(overwrite_factors=True);

In [ ]:
tca.rearrange_factors(
    undo_concat_complexDim=False,
    undo_concat_dictElements=False,
)

Save the `TCA` object in 'analysis_files' project directory.

In [ ]:
tca.save_config(path_config=path_config, overwrite=True, verbose=1)
tca.save_run_info(path_config=path_config, overwrite=True, verbose=1)
tca.save_run_data(path_config=path_config, overwrite=True, use_compression=False, verbose=1)

Clear some memory if needed. Useful if you ran the fit on a GPU. Optional.

In [ ]:
tca._cleanup()

In [ ]:
%matplotlib notebook
tca.plot_factors(
    figure_saver=None,
    show_figures=True,
)

# Visualize results

In [ ]:
import matplotlib.pyplot as plt

Load the `PointTracker`, `TCA`, and `VQT` data as dictionaries

In [ ]:
tca_data  = fr.h5_handling.simple_load(str(Path(directory_project) / 'analysis_files' / 'TCA.h5'), return_dict=True)
spec_data = fr.h5_handling.simple_load(str(Path(directory_project) / 'analysis_files' / 'VQT_Analyzer.h5'), return_dict=True)
pt_data   = fr.h5_handling.simple_load(str(Path(directory_project) / 'analysis_files' / 'PointTracker.h5'), return_dict=True)

visualize the spectral factors and corresponding spatial factors

In [ ]:
fig_tca_specFactor = plt.figure()
plt.plot(spec_data['frequencies'], (t:=tca_data['factors']['0']['frequency']) / t.max(axis=0))
plt.xscale('log')
plt.xlabel('frequency')
plt.legend(np.arange(tca_data['factors']['0']['frequency'].shape[1]) + 1);
plt.show()

Save figure

In [ ]:
figure_saver.save_figure(
    fig=fig_tca_specFactor,
    name_save=f'fig_tca_specFactor',
)

visualize the point factor magnitudes

In [ ]:
cmap = fr.helpers.simple_cmap(
    colors=[[0,0,0], [1,0,0], [1,1,0]], 
    over=[0,1,0], 
    under=[0,1,0], 
    bad=[0,1,0], 
)

factor_values = tca_data['factors_rearranged']['0']['(xy points)'].reshape(2, tca_data['factors_rearranged']['0']['(xy points)'].shape[0]//2, -1)
factor_magnitudes = np.abs(factor_values[0] + factor_values[1]*1j)
factor_angles = np.angle(factor_values[0] + factor_values[1]*1j)

pts_tmp = pt_data['point_positions']
im_tmp = videos.video_readers[0][0].cpu().numpy()

frame_visualizer = fr.visualization.FrameVisualizer(
    verbose=2,
    frame_height_width=videos.frame_height_width,
    point_sizes=6,
    alpha=0.6,
)

# point_colors = [fr.visualization.complex_colormap(m/m.max(), a).astype(np.int64) for m, a in zip(factor_magnitudes.T, factor_angles.T)]
# # point_colors = [(cmap(f/f.max())*255).astype(np.int64)[:,:3] for f in factor_magnitudes.T]
cmap = fr.helpers.Colorwheel()
point_colors = [cmap(a*2, (m/m.max())**2).astype(np.int64) for m, a in zip(factor_magnitudes.T, factor_angles.T)]

ims_factors_points = [frame_visualizer.visualize_image_with_points(image=im_tmp, points=pts_tmp, points_colors=[c]) for c in point_colors] 

fr.visualization.display_toggle_image_stack(ims_factors_points)

### Save images

Prepare an image saving helper class. It can save individual images in multiple formats as well as a list of images as a GIF.

In [ ]:
image_saver = fr.util.Image_Saver(
    path_config=path_config,
    dir_save=None,
    overwrite=True,
    verbose=1,
)

save individual factor images

In [ ]:
[image_saver.save_image(
    array_image=im,
    name_save=f'im_factors_points_{ii+1}',
    formats_save=['png'],
) for ii, im in enumerate(ims_factors_points)];

save gif of all the images

In [ ]:
image_saver.save_gif(
    array_images=fr.helpers.add_text_to_images(
        images=ims_factors_points,
        text=[[f'{ii+1}'] for ii in range(len(ims_factors_points))],
        position=(10, 40),
        font_size=1,
        line_width=2,
        color=(255,255,255),
    ),
    name_save=f'im_factors_points',
    frame_rate=1.0,
    loop=0,
);

# Demo playback

Playback a video with points overlayed.\
Make sure you have a `BufferedVideoReader` object called `videos` made of your videos

In [ ]:
idx_video_to_use = 0
# idx_frames_to_use = np.arange(9400, 88000)
idx_frames_to_use = np.arange(0, 88000)

videos.method_getitem = 'by_video'
# videos.method_getitem = 'continuous'

frame_visualizer = fr.visualization.FrameVisualizer(
# with fr.visualization.FrameVisualizer(
    display=True,
    error_checking=True,
    verbose=2,
    path_save=str(Path(directory_project) / 'visualizations' / 'point_tracking_demo.avi'),
#     path_save=None,
    frame_height_width=videos.frame_height_width,
    frame_rate=data.frame_rate,
    point_sizes=3,
    points_colors=(0, 0, 255),
#     points_colors=[(cmap((factor_magnitudes / factor_magnitudes.max(0)))*255)[:,:,:3].astype(int).transpose(1,0,2)[0]],
    alpha=0.6,
)

fr.visualization.play_video_with_points(
    bufferedVideoReader=videos[idx_video_to_use],
    frameVisualizer=frame_visualizer,
    points=list(pt_data['points_tracked'].values())[0],
    idx_frames=idx_frames_to_use,
)